# Logic trees for Nath & Thingbaijam (2012)

Read logic tree tables extracted from publication, along with a table of supplementary information write them to TEX and XML. Note that the logic trees require information from the source models, so if changes are made there then this must be rerun too.

In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
from IPython.display import display

%autoreload 2
import logic_tree_tools as ltt

from openquake.commonlib import nrml

In [3]:
# flag determines whether branch IDs are included in PDF drawings of logic trees
include_ids=False

## GMPE Logic Tree

In [4]:
ltt.nrml_to_pdf('example_gmpe_logic_tree.xml', include_ids=include_ids)

In [5]:
gsim_tree_tsv = 'gmpe_logic_tree.tsv'
gsim_tree_df = ltt.read_tree_tsv(gsim_tree_tsv)
gsim_tree_df.drop(['Tectonic Region Type', 'Qualifier', 'Short Names'], axis=1)

,uncertaintyType,applyToTectonicRegionType,uncertaintyModel
0,gmpeModel,active shallow crust normal,"[AkkarBommer2010, BooreAtkinson2008, CampbellB..."
1,gmpeModel,active shallow crust strike-slip reverse,"[AkkarBommer2010, BooreAtkinson2008, CampbellB..."
2,gmpeModel,intraplate margin lower,"[AtkinsonBoore2006, ToroEtAl2002, SharmaEtAl20..."
3,gmpeModel,intraplate margin upper,"[AtkinsonBoore2006, ToroEtAl2002, SharmaEtAl20..."
4,gmpeModel,stable shallow crust,"[AtkinsonBoore2006, ToroEtAl2002, Campbell2003..."
5,gmpeModel,subduction interface,"[AtkinsonBoore2003SInter, ZhaoEtAl2006SInter, ..."
6,gmpeModel,subduction interface megathrust,"[AtkinsonBoore2003SInter, AtkinsonMacias2009, ..."
7,gmpeModel,subduction intraslab Himalayas,"[AtkinsonBoore2003SSlabJapan, YoungsEtAl1997SS..."
8,gmpeModel,subduction intraslab,"[AtkinsonBoore2003SSlabCascadia, YoungsEtAl199..."


In [6]:
gsim_tree_deprecated_xml = gsim_tree_tsv.replace('.tsv','_deprecated.xml')
ltt.write_gsim_tree_nrml(gsim_tree_df, gsim_tree_deprecated_xml, validate=True)
gsim_tree_deprecated = nrml.read(gsim_tree_deprecated_xml)

<string>:2: DeprecationWarning: logic_tree_tools.write_gsim_tree_nrml has been deprecated. Use gsim_data_to_tree instead
/home/nick/src/python/GEM/oq-hazardlib/openquake/hazardlib/gsim/base.py:90: NotVerifiedWarning: RaghukanthIyengar2007 is not independently verified - the user is liable for their application
  warnings.warn(msg, NotVerifiedWarning)
/home/nick/src/python/GEM/oq-hazardlib/openquake/hazardlib/gsim/base.py:90: NotVerifiedWarning: Gupta2010SSlab is not independently verified - the user is liable for their application
  warnings.warn(msg, NotVerifiedWarning)


In [7]:
new_models = ['SharmaEtAl2009', 
              'Kanno2006Shallow', 'Kanno2006Deep', 
              'NathEtAl2012Lower', 'NathEtAl2012Upper', 
              'RaghukanthIyengar2007', 
              'Gupta2010SSlab',
              'AtkinsonBoore2003SSlabJapan','AtkinsonBoore2003SSlabCascadia']
gsim_tree_omit_new = ltt.df_to_tree(gsim_tree_df, omit=new_models, validate=True)
gsim_tree_omit_new_xml = gsim_tree_tsv.replace('.tsv','_omit_new.xml')
with open(gsim_tree_omit_new_xml, 'w+') as f:
    nrml.write([gsim_tree_omit_new], f, fmt='%g')
ltt.nrml_to_pdf(gsim_tree_omit_new_xml, include_ids=include_ids)

Omitting bs1 model 5 "SharmaEtAl2009" ...
Omitting bs1 model 4 "Kanno2006Shallow" ...
Omitting bs2 model 4 "Kanno2006Shallow" ...
Omitting bs3 model 4 "NathEtAl2012Lower" ...
Omitting bs3 model 3 "SharmaEtAl2009" ...
Omitting bs4 model 4 "NathEtAl2012Upper" ...
Omitting bs4 model 3 "SharmaEtAl2009" ...
Omitting bs5 model 4 "RaghukanthIyengar2007" ...
Omitting bs6 model 3 "Kanno2006Deep" ...
Omitting bs7 model 4 "Kanno2006Deep" ...
Omitting bs8 model 1 "AtkinsonBoore2003SSlabJapan" ...
Omitting bs9 model 4 "Gupta2010SSlab" ...
Omitting bs9 model 1 "AtkinsonBoore2003SSlabCascadia" ...


In [8]:
gsim_tree_read = nrml.read(gsim_tree_omit_new_xml)

In [9]:
# not clear why you don't get the same thing reading and writing
print gsim_tree_omit_new.logicTreeBranchingLevel.logicTreeBranchSet.logicTreeBranch.uncertaintyModel.text
print gsim_tree_read.logicTree.logicTreeBranchingLevel.logicTreeBranchSet.logicTreeBranch.uncertaintyModel.text.strip()
print gsim_tree_deprecated.logicTree.logicTreeBranchingLevel.logicTreeBranchSet.logicTreeBranch.uncertaintyModel.text.strip()

AkkarBommer2010
AkkarBommer2010
AkkarBommer2010


In [10]:
node = gsim_tree_omit_new[0][0]
print node
print '%s = %dx %s' %(ltt.strip_fqtag(node.tag), len(node), 
                      ltt.strip_fqtag(node[0].tag))
print ltt.get_dict_key_match(node.attrib,'id')

<logicTreeBranchSet {'branchSetID': 'bs1', 'applyToTectonicRegionType': 'active shallow crust normal', 'uncertaintyType': 'gmpeModel'} None ...>
logicTreeBranchSet = 3x logicTreeBranch
branchSetID


In [11]:
# let's also write an un-validated ("full") version of the tree
gsim_tree = ltt.df_to_tree(gsim_tree_df, validate=False)
gsim_tree_xml = gsim_tree_tsv.replace('.tsv','.xml')
with open(gsim_tree_xml, 'w+') as f:
    nrml.write([gsim_tree], f, fmt='%g')
ltt.nrml_to_pdf(gsim_tree_xml, include_ids=include_ids)

## Source Model Logic Tree

In [12]:
ltt.nrml_to_pdf('example_source_model_logic_tree.xml', include_ids=include_ids)

Too many (11) nodes in logicTree, abbreviating TEX to first & last 5


In [13]:
#source_tree_tsv_list = ['areal_model_logic_tree.tsv', 
#                        'source_model_logic_tree.tsv', 
#                        'recomputed_smoothed_model_logic_tree.tsv', 
#                        'smoothed_model_logic_tree.tsv']
source_tree_tsv_list = ['no_fmd_uncertainty.tsv']
for source_tree_tsv in source_tree_tsv_list:
    print "PROCESSING", source_tree_tsv
    source_tree_symbolic_df = ltt.read_tree_tsv(source_tree_tsv)
    display(source_tree_symbolic_df)
    
    source_tree_symbolic = ltt.df_to_tree(source_tree_symbolic_df, validate=True)
    source_tree_symbolic_xml = source_tree_tsv.replace('.tsv','_symbolic.xml')
    with open(source_tree_symbolic_xml, 'w+') as f:
        nrml.write([source_tree_symbolic], f, fmt='%g')
    ltt.nrml_to_pdf(source_tree_symbolic_xml, include_ids=include_ids)
        
    source_tree_df = ltt.df_expand_sources(source_tree_symbolic_df)
    display(pd.concat((source_tree_df.head(), source_tree_df.tail())))
    source_tree = ltt.df_to_tree(source_tree_df, validate=False)
    source_tree_xml = source_tree_tsv.replace('.tsv','.xml')
    with open(source_tree_xml, 'w+') as f:
        nrml.write([source_tree], f, fmt='%g')
    ltt.nrml_to_pdf(source_tree_xml, include_ids=include_ids)
    print

PROCESSING no_fmd_uncertainty.tsv


,uncertaintyType,applyToSources,uncertaintyModel,uncertaintyWeight
0,sourceModel,all,"[areal_source_model.xml, nath2012probabilistic...","[0.4, 0.27, 0.33]"


,uncertaintyType,applyToSources,uncertaintyModel,uncertaintyWeight
0,sourceModel,all,"[areal_source_model.xml, nath2012probabilistic...","[0.4, 0.27, 0.33]"
0,sourceModel,all,"[areal_source_model.xml, nath2012probabilistic...","[0.4, 0.27, 0.33]"


In [14]:
node = source_tree_symbolic[0][0]
print node
print '%s = %dx %s' %(ltt.strip_fqtag(node.tag), len(node), 
                      ltt.strip_fqtag(node[0].tag))
print ltt.get_dict_key_match(node.attrib,'id')
print node[0].uncertaintyModel

<logicTreeBranchSet {'branchSetID': 'bs1', 'uncertaintyType': 'sourceModel'} None ...>
logicTreeBranchSet = 3x logicTreeBranch
branchSetID
<uncertaintyModel {} areal_source_model.xml >


In [15]:
node = source_tree[0][0]
print node
print '%s = %dx %s' %(ltt.strip_fqtag(node.tag), len(node), 
                      ltt.strip_fqtag(node[0].tag))
print ltt.get_dict_key_match(node.attrib,'id')
model = node[0].uncertaintyModel
print model

<logicTreeBranchSet {'branchSetID': 'bs1', 'uncertaintyType': 'sourceModel'} None ...>
logicTreeBranchSet = 3x logicTreeBranch
branchSetID
<uncertaintyModel {} areal_source_model.xml >
